# Importing Python libraries

In [ ]:
import numpy as np  # library for numeric calculations   
import pandas as pd # library for reading data 
import seaborn as sns # library for plots 
from matplotlib import cm # library fo plots 
import matplotlib.pyplot as plt # library for plots 
from datetime import datetime # library to get the current time 
import datetime # library for time 
from sklearn.metrics import mean_squared_error # library for evaluation  mse 
from math import sqrt  # library for evaluation  rmse
from sklearn.metrics import mean_absolute_error # library for evaluation mae 
from sklearn.metrics import mean_absolute_percentage_error # library for evaluation  mape
from prettytable import PrettyTable # library for for comparison in tables 
from plotly.subplots import make_subplots # library for plots 
import plotly.graph_objs as go # library for plots 
from sklearn.linear_model import LinearRegression # library for linear regression model training 
import xgboost # library for xgboost model training 
from sklearn.ensemble import RandomForestRegressor # library for random forest model training 
from sklearn import linear_model # library for ridge model training 
from sklearn.neural_network import MLPRegressor # library for Mlp model training 
from sklearn.svm import SVR # library for SVR model training 
import re 
import string
from collections import Counter 
from sklearn import preprocessing
from matplotlib import rcParams
from sklearn.feature_extraction.text import TfidfVectorizer
import tweepy as tw 
from sklearn.neighbors import KNeighborsRegressor # library for KNN model training 
from statsmodels.tsa.statespace.sarimax import SARIMAX # library for Sarima model training 
from keras.models import Model # library for for LSTM model 
from keras.layers import LSTM, Activation, Input, Embedding # library for LSTM
from keras.layers import Activation # library for RNN 
from keras.models import Sequential # library for RNN 
from keras.layers import Dense, Conv1D, Flatten, Dropout # library for LSTM
import datetime as dt #library for dates 
from binance import Client #library for getting python binance data
from sklearn.model_selection import RandomizedSearchCV #library for seraching best parametr
from keras.wrappers.scikit_learn import KerasClassifier #library for neural network model
from sklearn.model_selection import GridSearchCV #library for searching best parameter
import statsmodels.api as smapi

from wordcloud import WordCloud
import nltk
from nltk import word_tokenize
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import spacy
from keras.preprocessing.text import Tokenizer
import matplotlib.cm as cm
from matplotlib import rcParams
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings # library to avoid system erros 
warnings.filterwarnings("ignore")  # library to avoid system erros 

# ------------------------------------------------------------------------------------------------------

## Settings

In [ ]:
cryptoTicker = "ETH"
cryptoPair = "USDT"

finalPair = cryptoTicker+cryptoPair

trainTestSplit = 0.8

# Loading dataset

In [ ]:
api_key = 'API' 
api_secret = 'SECRET_API'
client = Client(api_key, api_secret)
klines = client.get_historical_klines(finalPair, Client.KLINE_INTERVAL_4HOUR, "2021-01-01", "2022-01-01")
data = pd.DataFrame(klines)
data.columns = ['open_time','open', 'high', 'low', 'close', 'volume','close_time', 'qav','num_trades','taker_base_vol','taker_quote_vol', 'ignore']
data.index = [dt.datetime.fromtimestamp(x/1000.0) for x in data.close_time]
data=data[['close']]
data = data.iloc[:-1 , :]

# ------------------------------------------------------------------------------------------------------

# Exploratory Data analysis

# ------------------------------------------------------------------------------------------------------

### Five top records of data

In [ ]:
data.head() # showing the first 5 rows of data

### Five last records of data

In [ ]:
data.tail() # showing the last 5 rows of data

### Coloumns/features in data

In [ ]:
data.columns #columns in data

### Length of data

In [ ]:
print('lenght of data is', len(data)) #lenght of data

### Shape of data

In [ ]:
data.shape #shape of data

### Data information

In [ ]:
data.info() #information of data 

### Data types of all coloumns

In [ ]:
data.dtypes #data types of data

### Checking missing Values

In [ ]:
data[data.isnull().any(axis=1)].head() #checking the null values 

### Count of missing values

In [ ]:
np.sum(data.isnull().any(axis=1)) #checking the null values 

### Is there any missing values?

In [ ]:
data.isnull().values.any() #checking the null values 

### Counts of missing values in each column

In [ ]:
data.isnull().sum() #checking the null values columns wise 

### Data Description

In [ ]:
data.describe() # statics of data 

In [ ]:
data['close']=data['close'].astype(float)

### Spliting date into day, month and year

In [ ]:
data['date'] = data.index # taking dates in date column 
data['Month'] = data['date'].dt.month # extracting months from data
data['Day'] = data['date'].dt.day # extracting day from data
data['Year'] = data['date'].dt.year # extracting year from data
data['Hour'] = data['date'].dt.hour # extracting year from data
data['Minutes'] = data['date'].dt.minute # extracting year from data

### Close price with respect to year 

In [ ]:
sns.barplot(x='Year', y='close', data=data).set(title='Year vs Close') # plotting the comparison of year and close price
plt.show() #showing the plot 

### Close price with respect to Month 

In [ ]:
sns.barplot(x='Month', y='close', data=data).set(title='Year vs Close')  # plotting the comparison of month and close price
plt.show() #showing the plot 

### Close price with respect to Hour 

In [ ]:
sns.barplot(x='Hour', y='close', data=data).set(title='Hours vs Close')  # plotting the comparison of month and close price
plt.show() #showing the plot 

### Close price with respect to Minutes 

In [ ]:
sns.barplot(x='Minutes', y='close', data=data).set(title='Minutes vs Close')  # plotting the comparison of month and close price
plt.show() #showing the plot 

# ------------------------------------------------------------------------------------------------------

# Data Visualization

# ------------------------------------------------------------------------------------------------------

### CLOSE price values plot

In [ ]:
plt.figure(figsize=(20, 10)) #setting the width and height of the plot 
plt.subplot(1,1,1) #plot setting 
plt.plot(data.index, data['close'].values,color='orange',label='CLOSE') # plotting Close price 
plt.xlabel('Count of all') # x axis 
plt.ylabel('count of each record') # y axis 
plt.legend(loc='best') #legend name as Close 
plt.show() # showing the plot 

# ------------------------------------------------------------------------------------------------------

# Extracting the Indicators

# ------------------------------------------------------------------------------------------------------

### EMA and SMA 

### Moving average features

In [ ]:
data['EMA_50'] = data['close'].ewm(50).mean().shift() # calculting the ema 50
data['EMA_200'] =data['close'].ewm(200).mean().shift() # calculting the ema 200
data['MA_50'] = data['close'].rolling(50).mean() # calculting the ma 50
data['MA_200'] = data['close'].rolling(200).mean() # calculting the ma 200
fig = go.Figure() #plot setting 
fig.add_trace(go.Scatter(x=data.index, y=data.EMA_50, name='EMA 50')) # plotting the ema 50
fig.add_trace(go.Scatter(x=data.index, y=data.EMA_200, name='EMA 200')) # plotting the ema 200
fig.add_trace(go.Scatter(x=data.index, y=data.close, name='close', line_color='dimgray', opacity=0.3)) # plotting the close
fig.show() #showing the plot 

In [ ]:
fig = go.Figure() #plot setting 
fig.add_trace(go.Scatter(x=data.index, y=data.MA_50, name='MA 50')) # plotting the ma 50
fig.add_trace(go.Scatter(x=data.index, y=data.MA_200, name='MA 200')) # plotting the ma 200
fig.add_trace(go.Scatter(x=data.index, y=data.close, name='close', line_color='dimgray', opacity=0.3)) # plotting the close 
fig.show() # showing plot 

# Final Data after features extraction and preparation

In [ ]:
data=data.tail(len(data)-200) # ignoring the first 200 rows as there are Nan/missing values 
data #data 

# -----------------------------------------------------------------------------------------------------------

### Loading the whale data

In [ ]:
priceData=pd.read_csv('Datasets/dataFileWhale' + cryptoTicker + '.csv')

In [ ]:
li = priceData['tweet'].str.lower().str.cat(sep=' ')

# Getting Tokenization of the text

In [ ]:
priceData = priceData[['date','time','tweet']]
priceData['tweet'].head()

In [ ]:
#nums = re.findall(r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?(?!\d)', data['tweet'][0]
#data['amountTransfer'] = re.findall(r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?(?!\d)', data['tweet'])[0]
priceData['amountTransfer'] = priceData['tweet'].str.findall(r'\b\d{1,3}(?:,\d{3})*(?:\.\d+)?(?!\d)')
priceData['amountTransfer'] = priceData['amountTransfer'].str[1]

In [ ]:
data['whaleCount'] = 0

In [ ]:
for i, row in enumerate(priceData.itertuples(), 1):
    if row.time >= "00:00:00" and row.time < "04:00:00":
        if int(row.amountTransfer.replace(",","")) > 50000000:
            data.loc[data['date'] == (row.date + " " + "03:59:59" + ".999"), 'whaleCount'] += 2
        else:
            data.loc[data['date'] == (row.date + " " + "03:59:59" + ".999"), 'whaleCount'] += 1
    elif row.time >= "04:00:00" and row.time < "08:00:00":
        if int(row.amountTransfer.replace(",","")) > 50000000:
            data.loc[data['date'] == (row.date + " " + "07:59:59" + ".999"), 'whaleCount'] += 2
        else:
            data.loc[data['date'] == (row.date + " " + "07:59:59" + ".999"), 'whaleCount'] += 1
    elif row.time >= "08:00:00" and row.time < "12:00:00":
        if int(row.amountTransfer.replace(",","")) > 50000000:
            data.loc[data['date'] == (row.date + " " + "11:59:59" + ".999"), 'whaleCount'] += 2
        else:
            data.loc[data['date'] == (row.date + " " + "11:59:59" + ".999"), 'whaleCount'] += 1
    elif row.time >= "12:00:00" and row.time < "16:00:00":
        if int(row.amountTransfer.replace(",","")) > 50000000:
            data.loc[data['date'] == (row.date + " " + "15:59:59" + ".999"), 'whaleCount'] += 2
        else:
            data.loc[data['date'] == (row.date + " " + "15:59:59" + ".999"), 'whaleCount'] += 1
    elif row.time >= "16:00:00" and row.time < "20:00:00":
        if int(row.amountTransfer.replace(",","")) > 50000000:
            data.loc[data['date'] == (row.date + " " + "19:59:59" + ".999"), 'whaleCount'] += 2
        else:
            data.loc[data['date'] == (row.date + " " + "19:59:59" + ".999"), 'whaleCount'] += 1
    else:
        if int(row.amountTransfer.replace(",","")) > 50000000:
            data.loc[data['date'] == (row.date + " " + "23:59:59" + ".999"), 'whaleCount'] += 2
        else:
            data.loc[data['date'] == (row.date + " " + "23:59:59" + ".999"), 'whaleCount'] += 1

# -----------------------------------------------------------------------------------------------------------

### Loading the twitter data

In [ ]:
sentData=pd.read_csv('Datasets/dataFile' + cryptoTicker + '.csv')

In [ ]:
li = sentData['tweet'].str.lower().str.cat(sep=' ')

# Removing punctuation,numbers and returns list of words

In [ ]:
li2 = re.sub('[^A-Za-z]+', ' ', li)

# Removing the stopwords from the text

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')        
nltk_words = list(stopwords.words('english'))   
stop_words.extend(nltk_words)

# Getting Tokenization of the text

In [ ]:
word_tokens = word_tokenize(li2)
filtered_sentence = [w for w in word_tokens if not w in stop_words]
filtered_sentence = []
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)

# Removing characters which have length less than 2  

In [ ]:
without_single_chr = [word for word in filtered_sentence if len(word) > 2]

# Removing numbers

In [ ]:
cleaned_data_title = [word for word in without_single_chr if not word.isnumeric()]

# Frequency distribution of aspect words 

In [ ]:
word_dist = nltk.FreqDist(cleaned_data_title)
top100_words = pd.DataFrame(word_dist.most_common(100),
                    columns=['Word', 'Frequency'])

In [ ]:
plt.figure(figsize=(10,10))
sns.set_style("whitegrid")
ax = sns.barplot(x="Frequency",y="Word", data=top100_words.head(10))

# SentimentIntensityAnalyzer for getting sentiments

In [ ]:
sent_analyser = SentimentIntensityAnalyzer()
def sentiment(text):
    return (sent_analyser.polarity_scores(text)["compound"])

In [ ]:
sentData["Polarity"] = sentData["tweet"].apply(sentiment)

In [ ]:
sentData = sentData[['date','time','tweet','Polarity']]

In [ ]:
sentData.tail()

# Getting sentiments based on the popularity score

In [ ]:
def sentiments(data):
    if data['Polarity'] >= 0.05:
        val = "Positive"
    elif data['Polarity'] <= -0.05:
        val = "Negative"
    else:
        val = "Neutral"
    return val

In [ ]:
sentData['Sentiments'] = sentData.apply(sentiments, axis=1)

# sentiments with their counts

In [ ]:
sentData['Sentiments'].value_counts()

In [ ]:
sentData.iloc[:25]

In [ ]:
data['sentimentCount'] = 0

In [ ]:
for i, row in enumerate(sentData.itertuples(), 1):
    if row.time >= "00:00:00" and row.time < "04:00:00":
        if row.Sentiments == "Positive":
            data.loc[data['date'] == (row.date + " " + "03:59:59" + ".999"), 'sentimentCount'] += 1
        elif row.Sentiments == "Negative":
            data.loc[data['date'] == (row.date + " " + "03:59:59" + ".999"), 'sentimentCount'] -= 1
    elif row.time >= "04:00:00" and row.time < "08:00:00":
        if row.Sentiments == "Positive":
            data.loc[data['date'] == (row.date + " " + "07:59:59" + ".999"), 'sentimentCount'] += 1
        elif row.Sentiments == "Negative":
            data.loc[data['date'] == (row.date + " " + "07:59:59" + ".999"), 'sentimentCount'] -= 1
    elif row.time >= "08:00:00" and row.time < "12:00:00":
        if row.Sentiments == "Positive":
            data.loc[data['date'] == (row.date + " " + "11:59:59" + ".999"), 'sentimentCount'] += 1
        elif row.Sentiments == "Negative":
            data.loc[data['date'] == (row.date + " " + "11:59:59" + ".999"), 'sentimentCount'] -= 1
    elif row.time >= "12:00:00" and row.time < "16:00:00":
        if row.Sentiments == "Positive":
            data.loc[data['date'] == (row.date + " " + "15:59:59" + ".999"), 'sentimentCount'] += 1
        elif row.Sentiments == "Negative":
            data.loc[data['date'] == (row.date + " " + "15:59:59" + ".999"), 'sentimentCount'] -= 1
    elif row.time >= "16:00:00" and row.time < "20:00:00":
        if row.Sentiments == "Positive":
            data.loc[data['date'] == (row.date + " " + "19:59:59" + ".999"), 'sentimentCount'] += 1
        elif row.Sentiments == "Negative":
            data.loc[data['date'] == (row.date + " " + "19:59:59" + ".999"), 'sentimentCount'] -= 1
    else:
        if row.Sentiments == "Positive":
            data.loc[data['date'] == (row.date + " " + "23:59:59" + ".999"), 'sentimentCount'] += 1
        elif row.Sentiments == "Negative":
            data.loc[data['date'] == (row.date + " " + "23:59:59" + ".999"), 'sentimentCount'] -= 1

In [ ]:
data.head()

### Shift the data so future data is not used.

In [ ]:
data['whaleCount'] = data['whaleCount'].shift(1)
data['sentimentCount'] = data['sentimentCount'].shift(1)
data = data.iloc[1: , :]

# -------------------------------------------------------------------------------------------------------

In [ ]:
def adjustDatasetPredict(model):
    first90 = data[:int(trainTestSplit*(len(data)))] # training input features split 
    second10 = data[int(trainTestSplit*(len(data))):]  # training output split 

    second10['EMA_200'] = np.nan
    second10['MA_50'] = np.nan
    second10['MA_200'] = np.nan
    second10['close'] = np.nan
    second10.head()

    returnVal =[]
    for row in second10.iterrows():
        row[1]['EMA_50'] = first90['EMA_50'].iloc[-1]
        row[1]['EMA_200'] =first90['EMA_200'].iloc[-1]
        row[1]['MA_50'] = first90['MA_50'].iloc[-1]
        row[1]['MA_200'] = first90['MA_200'].iloc[-1]

        X_train=first90.drop(columns=['close','date']) #dropping the 'Close','date'
        y_train=first90['close'] # output close 
        newDataframe = pd.DataFrame().append([row[1]])
        X_test = newDataframe.drop(columns=['close','date'])

        prediction_test = model.predict(X_test)
        returnVal.append(prediction_test[0])
        row[1]['close'] = prediction_test[0]

        first90 = first90.append([row[1]])

        first90['EMA_50'] = first90['close'].ewm(50).mean().shift() # calculting the ema 50
        first90['EMA_200'] =first90['close'].ewm(200).mean().shift() # calculting the ema 200
        first90['MA_50'] = first90['close'].rolling(50).mean() # calculting the ma 50
        first90['MA_200'] = first90['close'].rolling(200).mean() # calculting the ma 200
    
    return returnVal

def adjustDatasetPredictSARIMAX(best_cfg):
    first90 = data[:int(trainTestSplit*(len(data)))] # training input features split 
    second10 = data[int(trainTestSplit*(len(data))):]  # training output split 
     
    second10['EMA_50'] = np.nan
    second10['EMA_200'] = np.nan
    second10['MA_50'] = np.nan
    second10['MA_200'] = np.nan
    second10['close'] = np.nan
    second10.head()

    returnVal =[]
    for row in second10.iterrows():
        row[1]['EMA_50'] = first90['EMA_50'].iloc[-1]
        row[1]['EMA_200'] =first90['EMA_200'].iloc[-1]
        row[1]['MA_50'] = first90['MA_50'].iloc[-1]
        row[1]['MA_200'] = first90['MA_200'].iloc[-1]
        
        newDataframe = pd.DataFrame().append([row[1]])
        newDataframe['close'].loc[:] = np.nan
        df6 = pd.concat([first90,newDataframe])

        model=smapi.tsa.statespace.SARIMAX(df6['close'],exog=df6[['EMA_50', 'EMA_200','MA_50','MA_200','sentimentCount','whaleCount']],order=(best_cfg),seasonal_order=(1,1,1,12))
        results=model.fit()
        
        preds = results.predict(start=len(df6), end=len(df6), exog=newDataframe[['EMA_50', 'EMA_200','MA_50','MA_200','sentimentCount','whaleCount']], dynamic = True)
        predSeries = pd.Series(preds.tolist())
        
        returnVal.append(predSeries[0])
        row[1]['close'] = predSeries[0]
        new90 = first90.append([row[1]])

        new90['EMA_50'] = new90['close'].ewm(50).mean().shift() # calculting the ema 50
        new90['EMA_200'] =new90['close'].ewm(200).mean().shift() # calculting the ema 200
        new90['MA_50'] = new90['close'].rolling(50).mean() # calculting the ma 50
        new90['MA_200'] = new90['close'].rolling(200).mean() # calculting the ma 200
        first90 = first90.append(new90.iloc[-1])
        print(first90.iloc[-1])
    return returnVal

def adjustDatasetPredictARIMAX(best_cfg):
    first90 = data[:int(trainTestSplit*(len(data)))] # training input features split 
    second10 = data[int(trainTestSplit*(len(data))):]  # training output split 
     
    second10['EMA_50'] = np.nan
    second10['EMA_200'] = np.nan
    second10['MA_50'] = np.nan
    second10['MA_200'] = np.nan
    second10['close'] = np.nan
    second10.head()

    returnVal =[]
    for row in second10.iterrows():
        row[1]['EMA_50'] = first90['EMA_50'].iloc[-1]
        row[1]['EMA_200'] =first90['EMA_200'].iloc[-1]
        row[1]['MA_50'] = first90['MA_50'].iloc[-1]
        row[1]['MA_200'] = first90['MA_200'].iloc[-1]
        
        newDataframe = pd.DataFrame().append([row[1]])
        newDataframe['close'].loc[:] = np.nan
        df6 = pd.concat([first90,newDataframe])

        model=smapi.tsa.statespace.SARIMAX(df6['close'],exog=df6[['EMA_50', 'EMA_200','MA_50','MA_200','sentimentCount','whaleCount']],order=(best_cfg))
        results=model.fit()
        
        preds = results.predict(start=len(df6), end=len(df6), exog=newDataframe[['EMA_50', 'EMA_200','MA_50','MA_200','sentimentCount','whaleCount']], dynamic = True)
        predSeries = pd.Series(preds.tolist())
        
        returnVal.append(predSeries[0])
        row[1]['close'] = predSeries[0]
        new90 = first90.append([row[1]])

        new90['EMA_50'] = new90['close'].ewm(50).mean().shift() # calculting the ema 50
        new90['EMA_200'] =new90['close'].ewm(200).mean().shift() # calculting the ema 200
        new90['MA_50'] = new90['close'].rolling(50).mean() # calculting the ma 50
        new90['MA_200'] = new90['close'].rolling(200).mean() # calculting the ma 200
        first90 = first90.append(new90.iloc[-1])
        print(first90.iloc[-1])
    return returnVal

<h1><center> Modelling </center></h1>

# -----------------------------------------------------------------------------------------------------------

### Taking input features 

In [ ]:
X=data.drop(columns=['close','date']) #dropping the 'Close','date'
X[X< 0] = 0 #making all negative values as 0
X #input features 

### Taking target as output

In [ ]:
y=data['close'] # output close 
y #output 

### Spliting the data into training and Testing

In [ ]:
X_train = X[:int(trainTestSplit*(len(X)))] # training input features split 
X_test = X[int(trainTestSplit*(len(X))):]  # testing input features split 

y_train = y[:int(trainTestSplit*(len(y)))]  # training output split 
y_test = y[int(trainTestSplit*(len(y))):] # testing output split 

# --------------------------------------------------------------------------------------------------

# ARIMAX Model

### Parametrs

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(priceData, arima_order):
    Xtest = priceData.drop(columns=['date','Month','Day','Year','Minutes','Hour'])
    train_data, test_data = Xtest.iloc[:int(len(Xtest['close'])*trainTestSplit)], Xtest.iloc[int(len(Xtest['close'])*trainTestSplit):]

    df5 = test_data.copy()
    df5['close'].loc[:] = np.nan

    df6 = pd.concat([train_data,df5])
    #exog = np.empty([df6['EMA_50'], df6['EMA_200'],df6['MA_50'],df6['MA_200']])
    model=smapi.tsa.statespace.SARIMAX(df6['close'],exog=df6[['EMA_50', 'EMA_200','MA_50','MA_200','sentimentCount','whaleCount']],order=arima_order)
    results=model.fit()

    preds =results.predict(start=len(train_data), end=len(train_data)+len(test_data)-1, dynamic = True)
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test_data['close'], preds.tolist()))

    return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(priceData, p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(priceData, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except Exception as e:
                    print(e)
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
    return best_cfg
 
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
best_cfg = evaluate_models(data, p_values, d_values, q_values)

In [ ]:
%%time
y_pred_test = adjustDatasetPredictARIMAX(best_cfg)

# Evaluation Measures on Test data

# MSE

In [ ]:
test_mse1=mean_squared_error(y_test,y_pred_test) # calculating the mse score
print('Mean Squared Error: ', round(test_mse1,5)) #pritng the results 

# MAE

In [ ]:
test_MAE1=mean_absolute_error(y_test,y_pred_test) # calculating the mae score
print('Mean Absolute Error: ', round(test_MAE1,5)) #pritng the results 

# RMSE

In [ ]:
test_rmse1=np.sqrt(test_mse1) # calculating the rmse score
print('Mean Squared Error: ', round(test_rmse1,5)) #pritng the results 

# MAPE

In [ ]:
test_mape1=mean_absolute_percentage_error(y_test,y_pred_test) # calculating the mape score
print('Mean Absolute Error: ', round(test_mape1,5)) #pritng the results 

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
df2 = pd.DataFrame({'train': y_train})
ax = df.plot()
df2.plot(ax=ax)
df.plot(kind='line',figsize=(18,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# --------------------------------------------------------------------------------------------------

# SARIMAX Model

### Parametrs

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(priceData, arima_order):
    Xtest = priceData.drop(columns=['date','Month','Day','Year','Minutes','Hour'])
    train_data, test_data = Xtest.iloc[:int(len(Xtest['close'])*trainTestSplit)], Xtest.iloc[int(len(Xtest['close'])*trainTestSplit):]

    df5 = test_data.copy()
    df5['close'].loc[:] = np.nan

    df6 = pd.concat([train_data,df5])
    #exog = np.empty([df6['EMA_50'], df6['EMA_200'],df6['MA_50'],df6['MA_200']])
    model=smapi.tsa.statespace.SARIMAX(df6['close'],exog=df6[['EMA_50', 'EMA_200','MA_50','MA_200','sentimentCount','whaleCount']],order=arima_order,seasonal_order=(1,1,1,12))
    results=model.fit()

    preds =results.predict(start=len(train_data), end=len(train_data)+len(test_data)-1, dynamic = True)
    # calculate out of sample error
    rmse = sqrt(mean_squared_error(test_data['close'], preds.tolist()))

    return rmse
 
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(priceData, p_values, d_values, q_values):
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p,d,q)
                try:
                    rmse = evaluate_arima_model(priceData, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except Exception as e:
                    print(e)
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))
    return best_cfg
 
# evaluate parameters
p_values = [0, 1, 2, 4, 6, 8, 10]
d_values = range(0, 3)
q_values = range(0, 3)
warnings.filterwarnings("ignore")
best_cfg = evaluate_models(data, p_values, d_values, q_values)

In [ ]:
%%time
y_pred_test = adjustDatasetPredictSARIMAX(best_cfg)

# Evaluation Measures on Test data

# MSE

In [ ]:
test_mse2=mean_squared_error(y_test,y_pred_test) # calculating the mse score
print('Mean Squared Error: ', round(test_mse2,5)) #pritng the results 

# MAE

In [ ]:
test_MAE2=mean_absolute_error(y_test,y_pred_test) # calculating the mae score
print('Mean Absolute Error: ', round(test_MAE2,5)) #pritng the results 

# RMSE

In [ ]:
test_rmse2=np.sqrt(test_mse2) # calculating the rmse score
print('Mean Squared Error: ', round(test_rmse2,5)) #pritng the results 

# MAPE

In [ ]:
test_mape2=mean_absolute_percentage_error(y_test,y_pred_test) # calculating the mape score
print('Mean Absolute Error: ', round(test_mape2,5)) #pritng the results 

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
df2 = pd.DataFrame({'train': y_train})
ax = df.plot()
df2.plot(ax=ax)
df.plot(kind='line',figsize=(18,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# --------------------------------------------------------------------------------------------------

# --------------------------------------------------------------------------------------------------

# MLP Regression Model

### Parametrs

In [ ]:
params = { 
    'max_iter': [100,200,300,400,500,600],
    'momentum':[0.7,0.8,0.9],
    'learning_rate_init':[0.001,0.002,0.004],
    'hidden_layer_sizes':[(100,),(300,),(200,)]
}

In [ ]:
mlp=MLPRegressor(verbose=True) # calling MLP model for training 
Model_R = RandomizedSearchCV(estimator = mlp, param_distributions = params, cv = 2) #calling search and it will help to get the best parametrs
Model_R.fit(X_train, y_train) # Training this model

### Best Parameter selected

In [ ]:
Model_R.best_params_

# Evaluation Measures on Test data

# MSE

In [ ]:
%%time
y_pred_test=adjustDatasetPredict(Model_R) # tetsing trained model on test data
test_mse3=mean_squared_error(y_test,y_pred_test) # calculating the mse score
print('Mean Squared Error: ', round(test_mse3,5)) #pritng the results 

# MAE

In [ ]:
test_MAE3=mean_absolute_error(y_test,y_pred_test) # calculating the mae score
print('Mean Absolute Error: ', round(test_MAE3,5)) #pritng the results 

# RMSE

In [ ]:
test_rmse3=np.sqrt(test_mse3) # calculating the rmse score
print('Mean Squared Error: ', round(test_rmse3,5)) #pritng the results 

# MAPE

In [ ]:
test_mape3=mean_absolute_percentage_error(y_test,y_pred_test) # calculating the mape score
print('Mean Absolute Error: ', round(test_mape3,5)) #pritng the results 

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
df2 = pd.DataFrame({'train': y_train})
ax = df.plot()
df2.plot(ax=ax)
df.plot(kind='line',figsize=(18,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# Ridge Regression Model

### Parametrs

In [ ]:
params = { 
    'alpha': [1.0,2.0,3.0,4.0],
    'max_iter':[100,200,300,400,500],
    'tol':[0.001,0.002,0.003,0.004]
}

In [ ]:
Ridge=linear_model.Ridge(alpha=4.0) # calling Ridge model for training 
Model_R = RandomizedSearchCV(estimator = Ridge, param_distributions = params, cv = 2) #calling search and it will help to get the best parametrs
Model_R.fit(X_train, y_train) # Training this model

### Best Parameter selected

In [ ]:
Model_R.best_params_

# Evaluation Measures on Test data

# MSE

In [ ]:
%%time
y_pred_test=adjustDatasetPredict(Model_R) # tetsing trained model on test data
test_mse7=mean_squared_error(y_test,y_pred_test) # calculating the mse score
print('Mean Squared Error: ', round(test_mse7,5)) #pritng the results 

# MAE

In [ ]:
test_MAE7=mean_absolute_error(y_test,y_pred_test) # calculating the mae score
print('Mean Absolute Error: ', round(test_MAE7,5)) #pritng the results 

# RMSE

In [ ]:
test_rmse7=np.sqrt(test_mse7) # calculating the rmse score
print('Mean Squared Error: ', round(test_rmse7,5)) #pritng the results 

# MAPE

In [ ]:
test_mape7=mean_absolute_percentage_error(y_test,y_pred_test) # calculating the mape score
print('Mean Absolute Error: ', round(test_mape7,5)) #pritng the results 

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
df2 = pd.DataFrame({'train': y_train})
ax = df.plot()
df2.plot(ax=ax)
df.plot(kind='line',figsize=(18,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# Random Forest Regression Model

### Parametrs

In [ ]:
params = { 
    'max_depth': [10,20,30,40,50],
    'random_state':[10,20,30,40],
    'min_samples_leaf':[10,20,30,40,50]
}

In [ ]:
RF=RandomForestRegressor() # calling RF model for training 
Model_R = RandomizedSearchCV(estimator = RF, param_distributions = params, cv = 2) #calling search and it will help to get the best parametrs
Model_R.fit(X_train, y_train) # Training this model

### Best Parameter selected

In [ ]:
Model_R.best_params_

# Evaluation Measures on Test data

# MSE

In [ ]:
%%time
y_pred_test=adjustDatasetPredict(Model_R) # tetsing trained model on test data
test_mse8=mean_squared_error(y_test,y_pred_test) # calculating the mse score
print('Mean Squared Error: ', round(test_mse8,5)) #pritng the results 

# MAE

In [ ]:
test_MAE8=mean_absolute_error(y_test,y_pred_test) # calculating the mae score
print('Mean Absolute Error: ', round(test_MAE8,5)) #pritng the results 

# RMSE

In [ ]:
test_rmse8=np.sqrt(test_mse8) # calculating the rmse score
print('Mean Squared Error: ', round(test_rmse8,5)) #pritng the results 

# MAPE

In [ ]:
test_mape8=mean_absolute_percentage_error(y_test,y_pred_test) # calculating the mape score
print('Mean Absolute Error: ', round(test_mape8,5)) #pritng the results 

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
df2 = pd.DataFrame({'train': y_train})
ax = df.plot()
df2.plot(ax=ax)
df.plot(kind='line',figsize=(18,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# XGBoost Model

### Parametrs

In [ ]:
params = { 
    'base_score': [0.1,0.2,0.3,0.4,0.5,0.6],
    'learning_rate':[0.1,0.2,0.3],
    'n_estimators':[100,200,300,400],
    'max_depth':[10,20,30,40,50]
    
}

In [ ]:
XG=xgboost.XGBRegressor() # calling Xgboost model for training 
Model_R = RandomizedSearchCV(estimator = XG, param_distributions = params, cv = 2) #calling search and it will help to get the best parametrs
Model_R.fit(X_train, y_train) # Training this model

### Best Parameter selected

In [ ]:
Model_R.best_params_

# Evaluation Measures on Test data

# MSE

In [ ]:
%%time
y_pred_test=adjustDatasetPredict(Model_R) # tetsing trained model on test data
test_mse9=mean_squared_error(y_test,y_pred_test) # calculating the mse score
print('Mean Squared Error: ', round(test_mse9,5)) #pritng the results 

# MAE

In [ ]:
test_MAE9=mean_absolute_error(y_test,y_pred_test) # calculating the mae score
print('Mean Absolute Error: ', round(test_MAE9,5)) #pritng the results 

# RMSE

In [ ]:
test_rmse9=np.sqrt(test_mse9) # calculating the rmse score
print('Mean Squared Error: ', round(test_rmse9,5)) #pritng the results 

# MAPE

In [ ]:
test_mape9=mean_absolute_percentage_error(y_test,y_pred_test) # calculating the mape score
print('Mean Absolute Error: ', round(test_mape9,5)) #pritng the results 

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
df2 = pd.DataFrame({'train': y_train})
ax = df.plot()
df2.plot(ax=ax)
df.plot(kind='line',figsize=(18,8))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

# Comparison of all algorithms Results on MSE score

In [ ]:
x = PrettyTable() #calling prettytable for the table 
print('\n') # for new line 
print("Comparison of all algorithms") # for  title 
x.field_names = ["Model", "MSE score"] # for  column names 

x.add_row(["ARIMAX Algorithm", round(test_mse1,3)]) # for the results of mse 
x.add_row(["SARIMAX Algorithm", round(test_mse2,3)]) # for the results of mse 
x.add_row(["MLP Algorithm", round(test_mse3,3)]) # for the results of mse 
x.add_row(["Ridge Algorithm", round(test_mse7,3)]) # for the results of mse 
x.add_row(["Random Forest Algorithm", round(test_mse8,3)]) # for the results of mse 
x.add_row(["XGB Algorithm", round(test_mse9,3)]) # for the results of mse 

print(x) # printing the table

# Comparison of all algorithms Results on MAE score

In [ ]:
x = PrettyTable() #calling prettytable for the table 
print('\n') # for new line 
print("Comparison of all algorithms") # for  title 
x.field_names = ["Model", "MAE score"] # for  column names 


x.add_row(["ARIMAX Algorithm", round(test_MAE1,3)]) # for the results of MAE 
x.add_row(["SARIMAX Algorithm", round(test_MAE2,3)]) # for the results of MAE 
x.add_row(["MLP Algorithm", round(test_MAE3,3)]) # for the results of MAE 
x.add_row(["Ridge Algorithm", round(test_MAE7,3)]) # for the results of MAE  
x.add_row(["Random Forest Algorithm", round(test_MAE8,3)]) # for the results of MAE 
x.add_row(["XGB Algorithm", round(test_MAE9,3)]) # for the results of MAE 

print(x) # printing the table

# Comparison of all algorithms Results on RMSE score

In [ ]:
x = PrettyTable() #calling prettytable for the table 
print('\n') # for new line 
print("Comparison of all algorithms") # for  title 
x.field_names = ["Model", "RMSE score"] # for  column names 

x.add_row(["ARIMAX Algorithm", round(test_rmse1,3)]) # for the results of rmse 
x.add_row(["SARIMAX Algorithm", round(test_rmse2,3)]) # for the results of rmse 
x.add_row(["MLP Algorithm", round(test_rmse3,3)]) # for the results of rmse 
x.add_row(["Ridge Algorithm", round(test_rmse7,3)]) # for the results of rmse 
x.add_row(["Random Forest Algorithm", round(test_rmse8,3)]) # for the results of rmse 
x.add_row(["XGB Algorithm", round(test_rmse9,3)]) # for the results of rmse 



print(x) # printing the table 
print('\n') #new line 

# Comparison of all algorithms Results on MAPE score

In [ ]:
x = PrettyTable() #calling prettytable for the table 
print('\n') # for new line 
print("Comparison of all algorithms") # for  title 
x.field_names = ["Model", "MAPE score"] # for  column names 

x.add_row(["ARIMAX Algorithm", round(test_mape1,3)]) # for the results of mape 
x.add_row(["SARIMAX Algorithm", round(test_mape2,3)]) # for the results of mape 
x.add_row(["MLP Algorithm", round(test_mape3,3)]) # for the results of mape 
x.add_row(["Ridge Algorithm", round(test_mape7,3)]) # for the results of mape 
x.add_row(["Random Forest Algorithm", round(test_mape8,3)]) # for the results of mape 
x.add_row(["XGB Algorithm", round(test_mape9,3)]) # for the results of mape 


print(x) # printing the table 
print('\n') #new line 